# Infer-5-Skills-IRT : Evaluation de Competences et Psychometrie

**Serie** : Programmation Probabiliste avec Infer.NET (5/13)  
**Duree estimee** : 60 minutes  
**Prerequis** : Infer-4-Bayesian-Networks

---

## Objectifs

- Comprendre les modeles d'evaluation cognitive
- Implementer le modele IRT (Item Response Theory) Difficulty-Ability
- Construire le modele DINA (Noisy-And) pour competences multiples
- Gerer les relations many-to-many entre competences et questions
- Estimer les parametres slip et guess

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-4-Bayesian-Networks](Infer-4-Bayesian-Networks.ipynb) | [Infer-6-TrueSkill](Infer-6-TrueSkill.ipynb) |

---

## 1. Configuration

Nous preparons l'environnement pour les modeles d'evaluation de competences. Ces modeles psychometriques (IRT, DINA) utilisent des variables latentes pour representer les capacites non observees des etudiants et les difficultes des questions.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


In [2]:
// Chargement du helper pour la visualisation des graphes de facteurs
#load "FactorGraphHelper.cs"

Console.WriteLine("FactorGraphHelper charge.");
Console.WriteLine($"Graphviz disponible : {FactorGraphHelper.IsGraphvizAvailable()}");

FactorGraphHelper charge.
Graphviz disponible : True


### Visualisation des graphes de facteurs

Le helper `FactorGraphHelper` permet d'afficher les graphes de facteurs generes par Infer.NET directement dans le notebook. Ces visualisations montrent :

- **Variables observees** (rectangles gris) : donnees fournies au modele
- **Variables latentes** (ovales) : parametres a inferer
- **Facteurs** (carres noirs) : relations probabilistes entre variables

Pour activer la generation des graphes, on utilise `engine.ShowFactorGraph = true` sur chaque moteur d'inference.

### Composants charges

| Package | Role |
|---------|------|
| `Microsoft.ML.Probabilistic` | Moteur d'inference principal |
| `Microsoft.ML.Probabilistic.Compiler` | Compilation des modeles en code C# |

Les modeles IRT et DINA utilisent des **variables latentes continues** (capacites) ou **discretes** (competences binaires) pour representer des caracteristiques non directement observables des etudiants.

## 2. Introduction a l'Evaluation Cognitive

### Probleme

Comment evaluer les competences d'un etudiant a partir de ses reponses a un test ?

### Defis

| Defi | Description |
|------|-------------|
| **Capacite latente** | La competence n'est pas directement observable |
| **Bruit** | Les reponses peuvent etre correctes par chance ou incorrectes par erreur |
| **Difficulte variable** | Les questions ont des difficultes differentes |
| **Competences multiples** | Une question peut necessiter plusieurs competences |

### Approches

| Modele | Caracteristique |
|--------|----------------|
| **IRT classique** | Capacite unidimensionnelle + difficulte des questions |
| **DINA** | Competences discretes multiples + matrice Q |
| **Bayesien hierarchique** | Parametres appris de maniere adaptative |

## 3. Modele IRT : Difficulty-Ability

### Formulation

$$P(\text{correct}_{ij}) = \sigma(\text{capacite}_i - \text{difficulte}_j)$$

Ou $\sigma$ est la fonction logistique (ou probit dans notre cas).

### Structure

```
capacite[i] ~ N(0, 1)    pour chaque etudiant
difficulte[j] ~ N(0, 1)  pour chaque question

avantage[i,j] = capacite[i] - difficulte[j]
reponse[i,j] ~ Probit(avantage[i,j])
```

In [3]:
// Modele IRT Difficulty-Ability

int nEtudiants = 10;
int nQuestions = 5;

// Donnees simulees : matrice de reponses (true = correct)
bool[,] reponses = new bool[,] {
    // Q1    Q2     Q3     Q4     Q5
    { true,  true,  true,  false, false },  // Etudiant 1 (bon)
    { true,  true,  false, false, false },  // Etudiant 2
    { true,  false, false, false, false },  // Etudiant 3 (faible)
    { true,  true,  true,  true,  false },  // Etudiant 4 (tres bon)
    { false, false, false, false, false },  // Etudiant 5 (tres faible)
    { true,  true,  false, true,  false },  // Etudiant 6
    { true,  true,  true,  false, true },   // Etudiant 7
    { true,  false, true,  false, false },  // Etudiant 8
    { true,  true,  true,  true,  true },   // Etudiant 9 (excellent)
    { false, true,  false, false, false }   // Etudiant 10
};

// Definition du modele
Range etudiant = new Range(nEtudiants).Named("etudiant");
Range question = new Range(nQuestions).Named("question");

// Capacites latentes des etudiants
VariableArray<double> capacite = Variable.Array<double>(etudiant).Named("capacite");
capacite[etudiant] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(etudiant);

// Difficultes des questions
VariableArray<double> difficulte = Variable.Array<double>(question).Named("difficulte");
difficulte[question] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(question);

// Discrimination (bruit)
Variable<double> discrimination = Variable.GammaFromShapeAndScale(2, 0.5).Named("discrimination");

// Reponses observees
VariableArray2D<bool> reponseVar = Variable.Array<bool>(etudiant, question).Named("reponse");

using (Variable.ForEach(etudiant))
{
    using (Variable.ForEach(question))
    {
        Variable<double> avantage = capacite[etudiant] - difficulte[question];
        Variable<double> avantageBruite = Variable.GaussianFromMeanAndPrecision(avantage, discrimination);
        reponseVar[etudiant, question] = (avantageBruite > 0);
    }
}

// Observations
reponseVar.ObservedValue = reponses;

Console.WriteLine("Modele IRT defini.");

Modele IRT defini.


### Structure des donnees

Les donnees sont organisees en une **matrice de reponses** (10 etudiants x 5 questions) :

| Etudiant | Q1 | Q2 | Q3 | Q4 | Q5 | Score |
|----------|----|----|----|----|----| ----- |
| E1 | T | T | T | F | F | 3/5 |
| E2 | T | T | F | F | F | 2/5 |
| E3 | T | F | F | F | F | 1/5 |
| E4 | T | T | T | T | F | 4/5 |
| E5 | F | F | F | F | F | 0/5 |
| E6 | T | T | F | T | F | 3/5 |
| E7 | T | T | T | F | T | 4/5 |
| E8 | T | F | T | F | F | 2/5 |
| E9 | T | T | T | T | T | 5/5 |
| E10 | F | T | F | F | F | 1/5 |

**Patterns interessants** :
- E1 et E6 ont le meme score mais des patterns differents (Q3 vs Q4)
- E7 reussit Q5 (difficile) mais pas Q4 → questions de difficulte similaire ?

### Lancement de l'inference

Le moteur d'inference va maintenant estimer simultanement :
- Les **capacites latentes** de chaque etudiant (variables non observees)
- Les **difficultes** de chaque question (egalement latentes)

L'algorithme utilise est **Expectation Propagation (EP)**, adapte aux modeles avec des variables continues et des observations binaires (reponses correctes/incorrectes).

> **Note technique** : L'inference conjointe de capacites et difficultes est possible car les observations (matrice de reponses) contraignent suffisamment le probleme. C'est le principe de la **calibration IRT** utilisee dans les tests standardises.

In [4]:
// Inference
InferenceEngine moteurIRT = new InferenceEngine(new ExpectationPropagation());
moteurIRT.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteurIRT.ShowFactorGraph = true;  // Genere un fichier .gv pour visualisation

Gaussian[] capacitePost = moteurIRT.Infer<Gaussian[]>(capacite);
Gaussian[] difficultePost = moteurIRT.Infer<Gaussian[]>(difficulte);

Console.WriteLine("=== Capacites des etudiants (IRT) ===");
for (int i = 0; i < nEtudiants; i++)
{
    int nCorrect = 0;
    for (int j = 0; j < nQuestions; j++) if (reponses[i, j]) nCorrect++;
    Console.WriteLine($"Etudiant {i+1} : capacite = {capacitePost[i].GetMean():F2} +/- {Math.Sqrt(capacitePost[i].GetVariance()):F2} (score: {nCorrect}/{nQuestions})");
}

Console.WriteLine("\n=== Difficultes des questions ===");
for (int j = 0; j < nQuestions; j++)
{
    int nReussi = 0;
    for (int i = 0; i < nEtudiants; i++) if (reponses[i, j]) nReussi++;
    Console.WriteLine($"Question {j+1} : difficulte = {difficultePost[j].GetMean():F2} (taux reussite: {nReussi}/{nEtudiants})");
}

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Capacites des etudiants (IRT) ===
Etudiant 1 : capacite = 0,26 +/- 0,60 (score: 3/5)
Etudiant 2 : capacite = -0,26 +/- 0,60 (score: 2/5)
Etudiant 3 : capacite = -0,81 +/- 0,62 (score: 1/5)
Etudiant 4 : capacite = 0,81 +/- 0,62 (score: 4/5)
Etudiant 5 : capacite = -1,44 +/- 0,67 (score: 0/5)
Etudiant 6 : capacite = 0,27 +/- 0,59 (score: 3/5)
Etudiant 7 : capacite = 0,81 +/- 0,61 (score: 4/5)
Etudiant 8 : capacite = -0,27 +/- 0,59 (score: 2/5)
Etudiant 9 : capacite = 1,44 +/- 0,67 (score: 5/5)
Etudiant 10 : capacite = -0,81 +/- 0,61 (score: 1/5)

=== Difficultes des questions ===
Question 1 : difficulte = -1,00 (taux reussite: 8/10)
Question 2 : difficulte = -0,64 (taux reussite: 7/10)
Question 3 : difficulte = 0,00 (taux reussite: 5/10)
Question 4 : difficulte = 0,64 (taux reussite: 3/10)
Question 5 : difficulte = 1,00 (taux reussite: 2/10)


In [5]:
// Visualisation du graphe de facteurs IRT
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_11_51_01.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 capacite[etudiant] 
 
<!-- node1 -->
 
 node1 
 
 Minus 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 a 
 
<!-- node3 -->
 
 node3 
 
 vdouble15[etudiant][question] 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 difficulte[question] 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 b 
 
<!-- node10 -->
 
 node10 
 
 Gaussian 
 
<!-- node3->node10 -->
 
 node3->node10 
 
 
 mean 
 
<!-- node4 -->
 
 node4 
 
 0 
 
<!-- node5 -->
 
 node5 
 
 Gaussian 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 mean 
 
<!-- node5->node2 -->
 
 node5->node2 
 
 
 
<!-- node6 -->
 
 node6 
 
 1 
 
<!-- node6->node5 -->
 
 node6->node5 
 
 
 precision 
 
<!-- node7 -->
 
 node7 
 
 0 
 
<!-- node8 -->
 
 node8 
 
 Gaussian 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 mean 
 
<!-- node8->node0 -->
 
 node8->node0 
 
 
 
<!-- node9 -->
 
 node9 
 
 1 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 precision 
 
<!-- node12 -->
 
 node12 
 
 vdouble16[etudiant][question] 
 
<!-- node10->node12 -->
 
 node10->node12 
 
 
 
<!-- node11 -->
 
 node11 
 
 discrimination 
 
<!-- node11->node10 -->
 
 node11->node10 
 
 
 precision 
 
<!-- node16 -->
 
 node16 
 
 IsPositive 
 
<!-- node12->node16 -->
 
 node12->node16 
 
 
 x 
 
<!-- node13 -->
 
 node13 
 
 2 
 
<!-- node14 -->
 
 node14 
 
 Sample 
 
<!-- node13->node14 -->
 
 node13->node14 
 
 
 shape 
 
<!-- node14->node11 -->
 
 node14->node11 
 
 
 
<!-- node15 -->
 
 node15 
 
 0,5 
 
<!-- node15->node14 -->
 
 node15->node14 
 
 
 scale 
 
<!-- node17 -->
 
 node17 
 
 reponse[etudiant,question] 
 
<!-- node16->node17 -->
 
 node16->node17


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Graphe de facteurs du modele IRT

Le graphe ci-dessus represente la structure du modele IRT :

| Element | Representation | Signification |
|---------|----------------|---------------|
| **capacite[i]** | Variable continue (ovale) | Capacite latente de chaque etudiant |
| **difficulte[j]** | Variable continue (ovale) | Difficulte de chaque question |
| **discrimination** | Variable continue | Parametre de bruit global |
| **reponse[i,j]** | Variable observee (rectangle) | Reponses correctes/incorrectes |
| **Facteurs** | Carres noirs | Relations probabilistes (Gaussian, IsPositive) |

La structure **bidimensionnelle** (etudiants x questions) cree une grille de facteurs connectant chaque capacite a chaque difficulte via les observations.

### Analyse détaillée des résultats IRT

**Observation clé** : La capacité estimée est **fortement corrélée** au score brut, mais pas parfaitement identique.

| Étudiant | Score | Capacité | Observation |
|----------|-------|----------|-------------|
| E5 | 0/5 | -1.44 | Le plus faible |
| E9 | 5/5 | +1.44 | Le plus fort |
| E1, E6 | 3/5 | ~+0.27 | Niveau moyen |

**Ordre des questions par difficulté** : Q1 < Q2 < Q3 < Q4 < Q5

Cela correspond aux taux de réussite observés (80%, 70%, 50%, 30%, 20%).

**Avantage du modèle IRT sur le score brut** :
- L'**écart-type** reflète l'incertitude (plus grande pour les scores intermédiaires)
- La capacité tient compte de la **difficulté des questions réussies**
- Un étudiant qui réussit Q5 (difficile) a une capacité plus élevée qu'un autre avec le même score mais sur des questions faciles

## 3bis. Évaluation du Modèle : Courbes ROC

Une **courbe ROC** (Receiver Operating Characteristic) permet d'évaluer la qualité des prédictions d'un modèle de classification. Pour l'IRT, nous pouvons prédire si un étudiant va répondre correctement à une question et comparer aux réponses réelles.

### Métriques d'évaluation

| Métrique | Formule | Interprétation |
|----------|---------|----------------|
| **AUC** | Aire sous la courbe ROC | 0.5 = aléatoire, 1.0 = parfait |
| **TPR** | TP / (TP + FN) | Taux de vrais positifs (sensibilité) |
| **FPR** | FP / (FP + TN) | Taux de faux positifs (1 - spécificité) |

### Calcul de la courbe ROC

Le code suivant calcule la courbe ROC en :
1. **Calculant P(correct)** pour chaque paire (etudiant, question) selon le modele
2. **Triant** les predictions par probabilite decroissante
3. **Calculant TPR/FPR** pour chaque seuil de classification
4. **Integrant** l'aire sous la courbe (AUC) par la methode des trapezes

> **Approximation probit-logit** : Le modele utilise une fonction probit, mais nous approximons avec une logistique (facteur 1.7) pour simplifier le calcul.

In [6]:
// Calcul de la courbe ROC pour le modele IRT

Console.WriteLine("=== Evaluation ROC du modele IRT ===\n");

// Calculer les probabilites de reponse correcte pour chaque paire (etudiant, question)
List<(double prob, bool actual)> predictions = new List<(double, bool)>();

for (int i = 0; i < nEtudiants; i++)
{
    for (int j = 0; j < nQuestions; j++)
    {
        // P(correct) approxime par la fonction probit
        double avantage = capacitePost[i].GetMean() - difficultePost[j].GetMean();
        // Approximation de Phi (CDF normale) par fonction logistique
        double probCorrect = 1.0 / (1.0 + Math.Exp(-1.7 * avantage));
        predictions.Add((probCorrect, reponses[i, j]));
    }
}

// Trier par probabilite decroissante
var sorted = predictions.OrderByDescending(p => p.prob).ToList();

// Calculer les points de la courbe ROC
int totalPos = sorted.Count(p => p.actual);
int totalNeg = sorted.Count - totalPos;

List<(double fpr, double tpr)> rocPoints = new List<(double, double)>();
rocPoints.Add((0.0, 0.0));

int tp = 0, fp = 0;
double lastProb = 1.0;

foreach (var pred in sorted)
{
    if (pred.actual) tp++;
    else fp++;
    
    double tpr = (double)tp / totalPos;
    double fpr = (double)fp / totalNeg;
    rocPoints.Add((fpr, tpr));
}

// Calcul de l'AUC (methode des trapezes)
double auc = 0;
for (int i = 1; i < rocPoints.Count; i++)
{
    double width = rocPoints[i].fpr - rocPoints[i-1].fpr;
    double height = (rocPoints[i].tpr + rocPoints[i-1].tpr) / 2;
    auc += width * height;
}

Console.WriteLine($"AUC (Area Under Curve) : {auc:F3}");
Console.WriteLine($"  0.5 = aleatoire, 1.0 = parfait");
Console.WriteLine($"  Interpretation : {(auc > 0.9 ? "Excellent" : auc > 0.8 ? "Bon" : auc > 0.7 ? "Acceptable" : "Faible")}");

// Afficher quelques points de la courbe
Console.WriteLine("\nPoints cles de la courbe ROC :");
Console.WriteLine("| Seuil | FPR  | TPR  |");
Console.WriteLine("|-------|------|------|");
double[] seuils = { 0.9, 0.7, 0.5, 0.3, 0.1 };
foreach (double seuil in seuils)
{
    int tpS = predictions.Count(p => p.prob >= seuil && p.actual);
    int fpS = predictions.Count(p => p.prob >= seuil && !p.actual);
    double tprS = (double)tpS / totalPos;
    double fprS = (double)fpS / totalNeg;
    Console.WriteLine($"| {seuil:F1}   | {fprS:F2} | {tprS:F2} |");
}

=== Evaluation ROC du modele IRT ===

AUC (Area Under Curve) : 0,949
  0.5 = aleatoire, 1.0 = parfait
  Interpretation : Excellent

Points cles de la courbe ROC :
| Seuil | FPR  | TPR  |
|-------|------|------|
| 0,9   | 0,00 | 0,28 |
| 0,7   | 0,00 | 0,64 |
| 0,5   | 0,16 | 0,84 |
| 0,3   | 0,36 | 1,00 |
| 0,1   | 0,72 | 1,00 |


### Interprétation de la Courbe ROC

**AUC (Area Under Curve)** mesure la capacité discriminante du modèle :

| AUC | Interprétation | Action suggérée |
|-----|----------------|-----------------|
| 0.9-1.0 | Excellent | Le modèle est très fiable |
| 0.8-0.9 | Bon | Utilisable en production |
| 0.7-0.8 | Acceptable | À améliorer si possible |
| 0.5-0.7 | Faible | Revoir le modèle |

**Lecture de la table** :
- **Seuil 0.7** : Si on prédit "correct" quand P(correct) > 0.7
  - TPR = % de vraies bonnes réponses capturées
  - FPR = % de fausses alertes (mauvaises réponses prédites correctes)

**Application pour l'IRT** :

Le modèle IRT devrait avoir une AUC élevée car :
1. Il capture la **vraie capacité** de chaque étudiant
2. Il tient compte de la **difficulté** de chaque question
3. La prédiction P(correct) = f(capacité - difficulté) est bien calibrée

> **Note** : Avec seulement 50 observations (10 étudiants × 5 questions), l'AUC peut être bruitée. Un test plus grand donnerait une estimation plus stable.

---

**Transition** : La courbe ROC nous montre que le modele IRT capture bien la structure des donnees. Cependant, l'IRT suppose une **capacite unidimensionnelle**. Pour des tests evaluant des competences distinctes, nous avons besoin du modele **DINA** presente dans la section suivante.

## 4. Modele DINA : Competences Discretes

### Motivation

Le modele IRT suppose une capacite **unidimensionnelle**. En realite, un test peut evaluer **plusieurs competences**.

### Modele DINA (Deterministic Input, Noisy And)

- Chaque etudiant possede ou non chaque competence (variable binaire)
- Chaque question necessite un sous-ensemble de competences (matrice Q)
- Reponse correcte si **toutes** les competences requises sont presentes
- Avec des erreurs slip et guess

### Parametres

| Parametre | Description |
|-----------|-------------|
| **Slip** | P(incorrect \| a toutes les competences) - erreur d'inattention |
| **Guess** | P(correct \| manque une competence) - reponse au hasard |

### Preparation des donnees DINA

Nous allons maintenant definir un scenario concret pour le modele DINA :

- **8 etudiants** avec differents profils de competences
- **6 questions** avec des exigences variees (1 a 3 competences)
- **3 competences** (C1, C2, C3)

Les donnees sont construites pour illustrer comment le modele discrimine entre les profils. Par exemple, un etudiant ayant C1 et C2 (mais pas C3) reussira Q1, Q2, Q4 mais echouera sur Q3, Q5, Q6.

In [7]:
// Modele DINA simplifie

int nEtud = 8;
int nQuest = 6;
int nCompetences = 3;

// Matrice Q : quelles competences sont requises pour chaque question
// Q1 : C1 seulement
// Q2 : C2 seulement
// Q3 : C3 seulement
// Q4 : C1 et C2
// Q5 : C2 et C3
// Q6 : C1, C2 et C3
bool[,] matriceQ = new bool[,] {
    // C1    C2     C3
    { true,  false, false },  // Q1
    { false, true,  false },  // Q2
    { false, false, true  },  // Q3
    { true,  true,  false },  // Q4
    { false, true,  true  },  // Q5
    { true,  true,  true  }   // Q6
};

// Donnees : reponses des etudiants
bool[,] repDINA = new bool[,] {
    // Q1    Q2     Q3     Q4     Q5     Q6
    { true,  true,  true,  true,  true,  true  },  // E1 : a tout
    { true,  true,  false, true,  false, false },  // E2 : C1, C2 seulement
    { true,  false, true,  false, false, false },  // E3 : C1, C3 seulement
    { false, true,  true,  false, true,  false },  // E4 : C2, C3 seulement
    { true,  false, false, false, false, false },  // E5 : C1 seulement
    { false, true,  false, false, false, false },  // E6 : C2 seulement
    { false, false, true,  false, false, false },  // E7 : C3 seulement
    { false, false, false, false, false, false }   // E8 : rien
};

Console.WriteLine("Donnees DINA definies.");
Console.WriteLine("\nMatrice Q (competences requises par question) :");
for (int q = 0; q < nQuest; q++)
{
    string req = "";
    for (int c = 0; c < nCompetences; c++)
        if (matriceQ[q, c]) req += $"C{c+1} ";
    Console.WriteLine($"  Q{q+1} : {req}");
}

Donnees DINA definies.

Matrice Q (competences requises par question) :
  Q1 : C1 
  Q2 : C2 
  Q3 : C3 
  Q4 : C1 C2 
  Q5 : C2 C3 
  Q6 : C1 C2 C3 


### Lecture de la matrice Q

La matrice Q definit la structure du test. Chaque ligne correspond a une question, chaque colonne a une competence.

| Question | C1 | C2 | C3 | Interpretation |
|----------|----|----|----|--------------| 
| Q1 | 1 | 0 | 0 | Evalue C1 seul |
| Q2 | 0 | 1 | 0 | Evalue C2 seul |
| Q3 | 0 | 0 | 1 | Evalue C3 seul |
| Q4 | 1 | 1 | 0 | Necessite C1 ET C2 |
| Q5 | 0 | 1 | 1 | Necessite C2 ET C3 |
| Q6 | 1 | 1 | 1 | Necessite les 3 competences |

> **Conception de test** : Les questions multi-competences (Q4, Q5, Q6) sont plus discriminantes mais aussi plus difficiles. Un bon test melange des questions a competence unique (pour le diagnostic) et des questions composites (pour valider la maitrise globale).

In [8]:
// Note sur l'implementation DINA complete
// Le modele DINA avec AND dynamique sur les competences requises est complexe a
// implementer dans Infer.NET en raison des limitations sur SetTo dans les blocs conditionnels.
// Nous utilisons ci-dessous une version simplifiee qui illustre les concepts.

// Affichage de la structure du modele DINA
Console.WriteLine("=== Structure du modele DINA ===");
Console.WriteLine();
Console.WriteLine("Pour chaque etudiant e et question q :");
Console.WriteLine("  1. Verifier si e a toutes les competences requises par q (selon matrice Q)");
Console.WriteLine("  2. Si oui : P(correct) = 1 - slip");
Console.WriteLine("  3. Si non : P(correct) = guess");
Console.WriteLine();
Console.WriteLine("Parametres a estimer :");
Console.WriteLine("  - Competences[e,c] : chaque etudiant a-t-il chaque competence ?");
Console.WriteLine("  - slip : probabilite d'erreur malgre les competences");
Console.WriteLine("  - guess : probabilite de reussite sans les competences");

=== Structure du modele DINA ===

Pour chaque etudiant e et question q :
  1. Verifier si e a toutes les competences requises par q (selon matrice Q)
  2. Si oui : P(correct) = 1 - slip
  3. Si non : P(correct) = guess

Parametres a estimer :
  - Competences[e,c] : chaque etudiant a-t-il chaque competence ?
  - slip : probabilite d'erreur malgre les competences
  - guess : probabilite de reussite sans les competences


### Limites de l'implementation dans Infer.NET

Le modele DINA complet necessite de calculer dynamiquement le ET logique sur un sous-ensemble variable de competences pour chaque question. Cette operation est difficile a exprimer dans Infer.NET en raison des restrictions sur `SetTo` dans les blocs conditionnels imbriques.

La cellule suivante presente une implementation simplifiee pour une question specifique, illustrant le raisonnement DINA.

### Implementation simplifiee pour une question

Pour illustrer le raisonnement DINA, nous implementons le modele pour une seule question (Q4) qui necessite deux competences (C1 ET C2). Cette approche permet de :

1. **Definir explicitement** la condition `aC1 & aC2`
2. **Observer** une reponse et voir comment cela affecte les probabilites des competences
3. **Estimer** les parametres slip et guess a partir des donnees

Le code utilise des **priors Beta(1,9)** pour slip et guess, centres sur des valeurs faibles (~0.1), ce qui correspond a l'hypothese que les erreurs d'inattention et les reponses au hasard sont relativement rares.

In [9]:
// Version simplifiee : modele DINA pour une question specifique

// Question Q4 necessite C1 ET C2
Variable<bool> aC1 = Variable.Bernoulli(0.5).Named("aC1");
Variable<bool> aC2 = Variable.Bernoulli(0.5).Named("aC2");

Variable<double> slipQ4 = Variable.Beta(1, 9).Named("slip");
Variable<double> guessQ4 = Variable.Beta(1, 9).Named("guess");

Variable<bool> toutesCompQ4 = (aC1 & aC2).Named("toutesComp");
Variable<bool> reponseQ4 = Variable.New<bool>().Named("reponse");

using (Variable.If(toutesCompQ4))
{
    reponseQ4.SetTo(!Variable.Bernoulli(slipQ4));
}
using (Variable.IfNot(toutesCompQ4))
{
    reponseQ4.SetTo(Variable.Bernoulli(guessQ4));
}

// Observation : l'etudiant a repondu correctement
reponseQ4.ObservedValue = true;

InferenceEngine mDINA = new InferenceEngine();
mDINA.Compiler.CompilerChoice = CompilerChoice.Roslyn;
mDINA.ShowFactorGraph = true;  // Genere un fichier .gv pour visualisation

Console.WriteLine("=== DINA : Inference des competences ===");
Console.WriteLine($"Question Q4 necessite C1 ET C2");
Console.WriteLine($"Observation : reponse correcte\n");
Console.WriteLine($"P(C1) = {mDINA.Infer<Bernoulli>(aC1).GetProbTrue():F3}");
Console.WriteLine($"P(C2) = {mDINA.Infer<Bernoulli>(aC2).GetProbTrue():F3}");
Console.WriteLine($"P(C1 ET C2) = {mDINA.Infer<Bernoulli>(toutesCompQ4).GetProbTrue():F3}");
Console.WriteLine($"\nSlip estime : {mDINA.Infer<Beta>(slipQ4).GetMean():F3}");
Console.WriteLine($"Guess estime : {mDINA.Infer<Beta>(guessQ4).GetMean():F3}");

=== DINA : Inference des competences ===
Question Q4 necessite C1 ET C2
Observation : reponse correcte

Compiling model...done.
P(C1) = 0,833
P(C2) = 0,833
P(C1 ET C2) = 0,750

Slip estime : 0,093
Guess estime : 0,120


In [10]:
// Visualisation du graphe de facteurs DINA simplifie
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_11_53_32.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Bernoulli(0,5) 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 aC1 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node3 -->
 
 node3 
 
 And 
 
<!-- node2->node3 -->
 
 node2->node3 
 
 
 a 
 
<!-- node5 -->
 
 node5 
 
 toutesComp 
 
<!-- node3->node5 -->
 
 node3->node5 
 
 
 
<!-- node4 -->
 
 node4 
 
 aC2 
 
<!-- node4->node3 -->
 
 node4->node3 
 
 
 b 
 
<!-- node12 -->
 
 node12 
 
 True 
 
<!-- node5->node12 -->
 
 node5->node12 
 
 
 condition 
 
<!-- node6 -->
 
 node6 
 
 Bernoulli(0,5) 
 
<!-- node7 -->
 
 node7 
 
 Random 
 
<!-- node6->node7 -->
 
 node6->node7 
 
 
 dist 
 
<!-- node7->node4 -->
 
 node7->node4 
 
 
 
<!-- node8 -->
 
 node8 
 
 slip 
 
<!-- node9 -->
 
 node9 
 
 Bernoulli 
 
<!-- node8->node9 -->
 
 node8->node9 
 
 
 probTrue 
 
<!-- node10 -->
 
 node10 
 
 vbool7 
 
<!-- node9->node10 -->
 
 node9->node10 
 
 
 
<!-- node11 -->
 
 node11 
 
 Not 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 b 
 
<!-- node11->node12 -->
 
 node11->node12 
 
 
 
<!-- node13 -->
 
 node13 
 
 guess 
 
<!-- node14 -->
 
 node14 
 
 Bernoulli 
 
<!-- node13->node14 -->
 
 node13->node14 
 
 
 probTrue 
 
<!-- node14->node12 -->
 
 node14->node12 
 
 
 
<!-- node15 -->
 
 node15 
 
 Beta(1,9)[mean=0,1] 
 
<!-- node16 -->
 
 node16 
 
 Random 
 
<!-- node15->node16 -->
 
 node15->node16 
 
 
 dist 
 
<!-- node16->node13 -->
 
 node16->node13 
 
 
 
<!-- node17 -->
 
 node17 
 
 Beta(1,9)[mean=0,1] 
 
<!-- node18 -->
 
 node18 
 
 Random 
 
<!-- node17->node18 -->
 
 node17->node18 
 
 
 dist 
 
<!-- node18->node8 -->
 
 node18->node8


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Graphe de facteurs du modele DINA simplifie

Le graphe illustre la structure conditionnelle du modele DINA :

| Element | Role |
|---------|------|
| **aC1, aC2** | Competences binaires (Bernoulli prior 0.5) |
| **toutesComp** | Conjonction aC1 AND aC2 |
| **slip, guess** | Parametres de bruit (Beta priors) |
| **reponse** | Variable observee (true = correct) |

**Structure conditionnelle** : La reponse depend de `toutesComp` via deux branches :
- Si `toutesComp=true` : P(correct) = 1 - slip
- Si `toutesComp=false` : P(correct) = guess

Cette structure en **gate** (porte conditionnelle) est caracteristique des modeles a competences discretes.

### Analyse du modèle DINA simplifié

**Résultats** :
- P(C1) = P(C2) = **0.833** (augmenté depuis le prior de 0.5)
- P(C1 ET C2) = **0.750**
- Slip estimé : ~0.09, Guess estimé : ~0.12

**Interprétation** :

L'observation d'une réponse correcte à Q4 (qui nécessite C1 ET C2) favorise les deux compétences :

1. **Sans guess** : La seule façon de répondre correctement est d'avoir C1 ET C2
2. **Avec guess** : Il y a une petite probabilité (~10%) de réussir sans les compétences

Le modèle infère que l'étudiant a **probablement** les deux compétences, mais conserve une incertitude résiduelle due au guess possible.

> **Remarque** : P(C1 ET C2) = 0.75 ≠ P(C1) × P(C2) = 0.69 car les observations créent une **dépendance** entre C1 et C2 (explaining away : si l'un est absent, l'autre doit être présent pour expliquer la réussite par guess).

## 5. Relations Many-to-Many

### Probleme

Dans le modele DINA, une question peut necessiter **plusieurs competences**, et une competence peut etre evaluee par **plusieurs questions**.

### Representation avec Subarray

Infer.NET permet d'utiliser `Variable.Subarray` pour extraire les competences requises pour chaque question.

### Demonstration avec plusieurs etudiants

Nous etendons maintenant le modele a **4 etudiants** et **2 questions** pour illustrer comment l'inference combine les observations de plusieurs sources :

- **Q1** : Necessite C1 seulement
- **Q2** : Necessite C1 ET C2

Les parametres slip (0.1) et guess (0.1) sont fixes pour simplifier l'inference sur les competences.

> **Objectif** : Montrer comment les reponses a Q1 et Q2 permettent de separer les etudiants qui ont C1 seul de ceux qui ont aussi C2.

In [11]:
// Demonstration avec plusieurs etudiants et questions

int nE = 4;
int nC = 3;

// Prior : chaque etudiant a 50% de chance d'avoir chaque competence
Range rE = new Range(nE).Named("etudiant");
Range rC = new Range(nC).Named("competence");

VariableArray2D<bool> competences = Variable.Array<bool>(rE, rC).Named("competences");
competences[rE, rC] = Variable.Bernoulli(0.5).ForEach(rE, rC);

// Simuler des observations sur plusieurs questions
// Q1 necessite C1 seulement
// Q2 necessite C1 ET C2

// Observations pour Q1 (C1 seulement)
bool[] obsQ1 = { true, true, false, true };  // E1, E2, E4 ont repondu correctement

for (int e = 0; e < nE; e++)
{
    Variable<bool> toutQ1 = competences[e, 0];  // Seulement C1
    Variable<bool> repQ1 = Variable.New<bool>().Named($"repQ1_E{e+1}");
    using (Variable.If(toutQ1))
    {
        repQ1.SetTo(Variable.Bernoulli(0.9));  // 1 - slip
    }
    using (Variable.IfNot(toutQ1))
    {
        repQ1.SetTo(Variable.Bernoulli(0.1));  // guess
    }
    repQ1.ObservedValue = obsQ1[e];
}

// Observations pour Q2 (C1 ET C2)
bool[] obsQ2 = { true, false, false, true };

for (int e = 0; e < nE; e++)
{
    Variable<bool> toutQ2 = competences[e, 0] & competences[e, 1];
    Variable<bool> repQ2 = Variable.New<bool>().Named($"repQ2_E{e+1}");
    using (Variable.If(toutQ2))
    {
        repQ2.SetTo(Variable.Bernoulli(0.9));
    }
    using (Variable.IfNot(toutQ2))
    {
        repQ2.SetTo(Variable.Bernoulli(0.1));
    }
    repQ2.ObservedValue = obsQ2[e];
}

InferenceEngine mMany = new InferenceEngine();
mMany.Compiler.CompilerChoice = CompilerChoice.Roslyn;
mMany.ShowFactorGraph = true;  // Genere un fichier .gv pour visualisation

Bernoulli[,] compPost = mMany.Infer<Bernoulli[,]>(competences);

Console.WriteLine("=== Inference des competences (many-to-many) ===");
Console.WriteLine("Q1 necessite C1, Q2 necessite C1 ET C2\n");
Console.WriteLine("Observations : Q1=[T,T,F,T], Q2=[T,F,F,T]\n");

for (int e = 0; e < nE; e++)
{
    Console.Write($"Etudiant {e+1} : ");
    for (int c = 0; c < nC; c++)
    {
        Console.Write($"C{c+1}={compPost[e,c].GetProbTrue():F2} ");
    }
    Console.WriteLine();
}

Compiling model...done.
=== Inference des competences (many-to-many) ===
Q1 necessite C1, Q2 necessite C1 ET C2

Observations : Q1=[T,T,F,T], Q2=[T,F,F,T]

Etudiant 1 : C1=0,98 C2=0,89 C3=0,50 
Etudiant 2 : C1=0,83 C2=0,17 C3=0,50 
Etudiant 3 : C1=0,06 C2=0,48 C3=0,50 
Etudiant 4 : C1=0,98 C2=0,89 C3=0,50 


In [12]:
// Visualisation du graphe de facteurs many-to-many
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_11_54_04.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 competences[vint15,vint16] 
 
<!-- node1 -->
 
 node1 
 
 And 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 a 
 
<!-- node3 -->
 
 node3 
 
 vbool32 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 competences[vint17,vint18] 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 b 
 
<!-- node33 -->
 
 node33 
 
 True 
 
<!-- node3->node33 -->
 
 node3->node33 
 
 
 condition 
 
<!-- node4 -->
 
 node4 
 
 competences[vint19,vint20] 
 
<!-- node5 -->
 
 node5 
 
 And 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 a 
 
<!-- node7 -->
 
 node7 
 
 vbool38 
 
<!-- node5->node7 -->
 
 node5->node7 
 
 
 
<!-- node6 -->
 
 node6 
 
 competences[vint21,vint22] 
 
<!-- node6->node5 -->
 
 node6->node5 
 
 
 b 
 
<!-- node28 -->
 
 node28 
 
 False 
 
<!-- node7->node28 -->
 
 node7->node28 
 
 
 condition 
 
<!-- node8 -->
 
 node8 
 
 competences[vint23,vint24] 
 
<!-- node9 -->
 
 node9 
 
 And 
 
<!-- node8->node9 -->
 
 node8->node9 
 
 
 a 
 
<!-- node11 -->
 
 node11 
 
 vbool44 
 
<!-- node9->node11 -->
 
 node9->node11 
 
 
 
<!-- node10 -->
 
 node10 
 
 competences[vint25,vint26] 
 
<!-- node10->node9 -->
 
 node10->node9 
 
 
 b 
 
<!-- node23 -->
 
 node23 
 
 False 
 
<!-- node11->node23 -->
 
 node11->node23 
 
 
 condition 
 
<!-- node12 -->
 
 node12 
 
 competences[vint27,vint28] 
 
<!-- node13 -->
 
 node13 
 
 And 
 
<!-- node12->node13 -->
 
 node12->node13 
 
 
 a 
 
<!-- node15 -->
 
 node15 
 
 vbool50 
 
<!-- node13->node15 -->
 
 node13->node15 
 
 
 
<!-- node14 -->
 
 node14 
 
 competences[vint29,vint30] 
 
<!-- node14->node13 -->
 
 node14->node13 
 
 
 b 
 
<!-- node18 -->
 
 node18 
 
 True 
 
<!-- node15->node18 -->
 
 node15->node18 
 
 
 condition 
 
<!-- node16 -->
 
 node16 
 
 Bernoulli(0,9) 
 
<!-- node17 -->
 
 node17 
 
 Random 
 
<!-- node16->node17 -->
 
 node16->node17 
 
 
 dist 
 
<!-- node17->node18 -->
 
 node17->node18 
 
 
 
<!-- node19 -->
 
 node19 
 
 Bernoulli(0,1) 
 
<!-- node20 -->
 
 node20 
 
 Random 
 
<!-- node19->node20 -->
 
 node19->node20 
 
 
 dist 
 
<!-- node20->node18 -->
 
 node20->node18 
 
 
 
<!-- node21 -->
 
 node21 
 
 Bernoulli(0,9) 
 
<!-- node22 -->
 
 node22 
 
 Random 
 
<!-- node21->node22 -->
 
 node21->node22 
 
 
 dist 
 
<!-- node22->node23 -->
 
 node22->node23 
 
 
 
<!-- node24 -->
 
 node24 
 
 Bernoulli(0,1) 
 
<!-- node25 -->
 
 node25 
 
 Random 
 
<!-- node24->node25 -->
 
 node24->node25 
 
 
 dist 
 
<!-- node25->node23 -->
 
 node25->node23 
 
 
 
<!-- node26 -->
 
 node26 
 
 Bernoulli(0,9) 
 
<!-- node27 -->
 
 node27 
 
 Random 
 
<!-- node26->node27 -->
 
 node26->node27 
 
 
 dist 
 
<!-- node27->node28 -->
 
 node27->node28 
 
 
 
<!-- node29 -->
 
 node29 
 
 Bernoulli(0,1) 
 
<!-- node30 -->
 
 node30 
 
 Random 
 
<!-- node29->node30 -->
 
 node29->node30 
 
 
 dist 
 
<!-- node30->node28 -->
 
 node30->node28 
 
 
 
<!-- node31 -->
 
 node31 
 
 Bernoulli(0,9) 
 
<!-- node32 -->
 
 node32 
 
 Random 
 
<!-- node31->node32 -->
 
 node31->node32 
 
 
 dist 
 
<!-- node32->node33 -->
 
 node32->node33 
 
 
 
<!-- node34 -->
 
 node34 
 
 Bernoulli(0,1) 
 
<!-- node35 -->
 
 node35 
 
 Random 
 
<!-- node34->node35 -->
 
 node34->node35 
 
 
 dist 
 
<!-- node35->node33 -


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Graphe de facteurs many-to-many

Ce graphe montre la structure **many-to-many** caracteristique des modeles DINA :

| Aspect | Observation |
|--------|-------------|
| **Matrice competences[4,3]** | 12 variables binaires (4 etudiants x 3 competences) |
| **Reponses Q1** | Connectees uniquement a C1 (1 competence requise) |
| **Reponses Q2** | Connectees a C1 AND C2 via facteurs conjonctifs |
| **C3** | Non connecte aux observations (reste au prior 0.5) |

**Pattern de connexion** :
- Chaque **etudiant** a ses propres variables de competences
- Chaque **question** connecte les competences requises aux observations
- Les facteurs **AND** implementent la logique "toutes les competences necessaires"

Cette structure permet l'inference **independante** des profils de competences pour chaque etudiant.

### Analyse des inférences many-to-many

**Pattern des résultats** :

| Étudiant | Q1(C1) | Q2(C1∧C2) | C1 inféré | C2 inféré | Interprétation |
|----------|--------|-----------|-----------|-----------|----------------|
| E1 | ✓ | ✓ | **0.98** | **0.89** | A probablement les deux |
| E2 | ✓ | ✗ | 0.83 | **0.17** | A C1, pas C2 |
| E3 | ✗ | ✗ | **0.06** | 0.48 | N'a pas C1 |
| E4 | ✓ | ✓ | **0.98** | **0.89** | A probablement les deux |

**Points clés** :

1. **C3 reste à 0.50** pour tous les étudiants car aucune question ne l'évalue
2. **E2** : Réussit Q1 (C1 seul) mais échoue Q2 (C1∧C2) → P(C2) chute à 0.17
3. **E1 vs E4** : Mêmes observations, mêmes inférences (cohérence du modèle)

> **Puissance du modèle** : Les questions multi-compétences (Q2) permettent de **discriminer** entre les compétences mieux qu'avec des questions à compétence unique.

## 6. Estimation des Parametres Slip/Guess

### Objectif

Apprendre les parametres slip et guess a partir des donnees.

### Approche

- Utiliser des priors Beta sur slip et guess
- L'inference met a jour ces distributions

### Simulation de donnees pour l'estimation

Pour evaluer la capacite du modele a estimer slip et guess, nous :

1. **Fixons** les vraies valeurs : slip=0.1, guess=0.2, P(competence)=0.6
2. **Simulons** 20 observations selon le modele generatif
3. **Inferons** les parametres a partir des observations seules (sans connaitre les vraies competences)

Cette approche permet de comparer les estimations aux vraies valeurs et d'evaluer le **biais** eventuel de l'inference.

> **Defi** : L'estimation est difficile car les competences individuelles sont **non observees**. Le modele doit les inferer en meme temps que slip et guess.

In [13]:
// Estimation de slip/guess avec donnees multiples

// Simuler des donnees ou on connait les vraies competences
int nObs = 20;
Random rng = new Random(42);

double vraiSlip = 0.1;
double vraiGuess = 0.2;

// Generer des donnees
bool[] vraiComp = new bool[nObs];    // Vrai etat de competence
bool[] obsRep = new bool[nObs];       // Reponse observee

for (int i = 0; i < nObs; i++)
{
    vraiComp[i] = rng.NextDouble() < 0.6;  // 60% ont la competence
    if (vraiComp[i])
    {
        obsRep[i] = rng.NextDouble() > vraiSlip;  // Correct sauf slip
    }
    else
    {
        obsRep[i] = rng.NextDouble() < vraiGuess;  // Incorrect sauf guess
    }
}

// Modele pour estimer slip et guess
Variable<double> slipEst = Variable.Beta(1, 1).Named("slip");
Variable<double> guessEst = Variable.Beta(1, 1).Named("guess");
Variable<double> pComp = Variable.Beta(1, 1).Named("pComp");

Range rObs = new Range(nObs).Named("observation");
VariableArray<bool> comp = Variable.Array<bool>(rObs).Named("competence");
VariableArray<bool> rep = Variable.Array<bool>(rObs).Named("reponse");

comp[rObs] = Variable.Bernoulli(pComp).ForEach(rObs);

using (Variable.ForEach(rObs))
{
    using (Variable.If(comp[rObs]))
    {
        rep[rObs] = !Variable.Bernoulli(slipEst);
    }
    using (Variable.IfNot(comp[rObs]))
    {
        rep[rObs] = Variable.Bernoulli(guessEst);
    }
}

rep.ObservedValue = obsRep;

InferenceEngine mSlipGuess = new InferenceEngine(new ExpectationPropagation());
mSlipGuess.Compiler.CompilerChoice = CompilerChoice.Roslyn;
mSlipGuess.ShowFactorGraph = true;  // Genere un fichier .gv pour visualisation

Console.WriteLine("=== Estimation Slip/Guess ===");
Console.WriteLine($"Vraies valeurs : slip={vraiSlip}, guess={vraiGuess}\n");
Console.WriteLine($"Estimations :");
Console.WriteLine($"  Slip : {mSlipGuess.Infer<Beta>(slipEst)}");
Console.WriteLine($"  Guess : {mSlipGuess.Infer<Beta>(guessEst)}");
Console.WriteLine($"  P(competence) : {mSlipGuess.Infer<Beta>(pComp)}");

=== Estimation Slip/Guess ===
Vraies valeurs : slip=0,1, guess=0,2

Estimations :
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
  Slip : Beta(3,871,7,698)[mean=0,3346]
  Guess : Beta(7,698,3,871)[mean=0,6654]
  P(competence) : Beta(1,1)[mean=0,5]


### Analyse critique des estimations

**Observation** : Les estimations divergent significativement des vraies valeurs :

| Parametre | Vraie valeur | Estimation | Ecart |
|-----------|--------------|------------|-------|
| Slip | 0.10 | ~0.33 | +0.23 |
| Guess | 0.20 | ~0.67 | +0.47 |
| P(comp) | 0.60 | 0.50 | -0.10 |

**Pourquoi cette divergence ?**

1. **Probleme d'identifiabilite** : Sans connaitre les vraies competences, le modele ne peut pas distinguer :
   - Un etudiant competent qui fait une erreur (slip)
   - Un etudiant incompetent qui devine correctement (guess)

2. **Symetrie du probleme** : Noter que slip + guess ≈ 1.0, ce qui suggere que le modele "inverse" partiellement l'interpretation.

3. **Taille d'echantillon** : 20 observations sont insuffisantes pour estimer 3 parametres latents de maniere fiable.

> **Lecon importante** : L'estimation de slip/guess **sans information supplementaire** sur les vraies competences est un probleme mal pose. En pratique, on utilise :
> - Des **ancres** (items dont on connait la difficulte)
> - Des **priors informatifs** bases sur des etudes pilotes
> - Des **contraintes** (ex: slip < 0.3, guess < 0.3)


### Graphe de facteurs pour l'estimation slip/guess

Ce graphe illustre le probleme d'**identifiabilite** dans l'estimation des parametres de bruit :

| Variable | Type | Role |
|----------|------|------|
| **slip** | Continue (Beta) | Parametre global d'erreur d'inattention |
| **guess** | Continue (Beta) | Parametre global de reponse au hasard |
| **pComp** | Continue (Beta) | Probabilite globale d'avoir la competence |
| **competence[i]** | Binaire latent | Competence de chaque individu |
| **reponse[i]** | Binaire observe | Reponse de chaque individu |

**Probleme visible** : Les parametres slip, guess et pComp sont **tous trois connectes** a chaque observation via les competences latentes. Sans information supplementaire, le modele ne peut pas distinguer les differentes sources de bruit.

In [14]:
// Visualisation du graphe de facteurs pour l'estimation slip/guess
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_11_54_84.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Beta(1,1)[mean=0,5] 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 slip 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node3 -->
 
 node3 
 
 Bernoulli 
 
<!-- node2->node3 -->
 
 node2->node3 
 
 
 probTrue 
 
<!-- node4 -->
 
 node4 
 
 vbool59[observation] 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 
<!-- node5 -->
 
 node5 
 
 Not 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 b 
 
<!-- node6 -->
 
 node6 
 
 reponse[observation] 
 
<!-- node5->node6 -->
 
 node5->node6 
 
 
 
<!-- node7 -->
 
 node7 
 
 competence[observation] 
 
<!-- node7->node6 -->
 
 node7->node6 
 
 
 condition 
 
<!-- node8 -->
 
 node8 
 
 guess 
 
<!-- node9 -->
 
 node9 
 
 Bernoulli 
 
<!-- node8->node9 -->
 
 node8->node9 
 
 
 probTrue 
 
<!-- node9->node6 -->
 
 node9->node6 
 
 
 
<!-- node10 -->
 
 node10 
 
 Beta(1,1)[mean=0,5] 
 
<!-- node11 -->
 
 node11 
 
 Random 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 dist 
 
<!-- node11->node8 -->
 
 node11->node8 
 
 
 
<!-- node12 -->
 
 node12 
 
 pComp 
 
<!-- node13 -->
 
 node13 
 
 Bernoulli 
 
<!-- node12->node13 -->
 
 node12->node13 
 
 
 probTrue 
 
<!-- node13->node7 -->
 
 node13->node7 
 
 
 
<!-- node14 -->
 
 node14 
 
 Beta(1,1)[mean=0,5] 
 
<!-- node15 -->
 
 node15 
 
 Random 
 
<!-- node14->node15 -->
 
 node14->node15 
 
 
 dist 
 
<!-- node15->node12 -->
 
 node15->node12


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime




### Version amelioree avec priors informatifs

Voici comment contraindre les estimations avec des priors realistes :

In [15]:
// Version avec priors informatifs pour slip/guess

// Memes donnees simulees
Variable<double> slipInf = Variable.Beta(2, 18).Named("slipInformatif");  // Prior centre sur 0.1
Variable<double> guessInf = Variable.Beta(3, 12).Named("guessInformatif");  // Prior centre sur 0.2
Variable<double> pCompInf = Variable.Beta(6, 4).Named("pCompInformatif");  // Prior centre sur 0.6

Range rObs2 = new Range(nObs).Named("observation2");
VariableArray<bool> comp2 = Variable.Array<bool>(rObs2).Named("competence2");
VariableArray<bool> rep2 = Variable.Array<bool>(rObs2).Named("reponse2");

comp2[rObs2] = Variable.Bernoulli(pCompInf).ForEach(rObs2);

using (Variable.ForEach(rObs2))
{
    using (Variable.If(comp2[rObs2]))
    {
        rep2[rObs2] = !Variable.Bernoulli(slipInf);
    }
    using (Variable.IfNot(comp2[rObs2]))
    {
        rep2[rObs2] = Variable.Bernoulli(guessInf);
    }
}

rep2.ObservedValue = obsRep;

InferenceEngine mInf = new InferenceEngine(new ExpectationPropagation());
mInf.Compiler.CompilerChoice = CompilerChoice.Roslyn;
mInf.ShowFactorGraph = true;  // Genere un fichier .gv pour visualisation

Console.WriteLine("=== Estimation avec Priors Informatifs ===");
Console.WriteLine($"Vraies valeurs : slip={vraiSlip}, guess={vraiGuess}, P(comp)=0.6\n");
Console.WriteLine($"Priors : Beta(2,18) pour slip, Beta(3,12) pour guess, Beta(6,4) pour P(comp)\n");
Console.WriteLine($"Estimations :");
var slipPost = mInf.Infer<Beta>(slipInf);
var guessPost = mInf.Infer<Beta>(guessInf);
var pCompPost = mInf.Infer<Beta>(pCompInf);
Console.WriteLine($"  Slip : moyenne={slipPost.GetMean():F3} (vraie: {vraiSlip})");
Console.WriteLine($"  Guess : moyenne={guessPost.GetMean():F3} (vraie: {vraiGuess})");
Console.WriteLine($"  P(competence) : moyenne={pCompPost.GetMean():F3} (vraie: 0.6)");

=== Estimation avec Priors Informatifs ===
Vraies valeurs : slip=0,1, guess=0,2, P(comp)=0.6

Priors : Beta(2,18) pour slip, Beta(3,12) pour guess, Beta(6,4) pour P(comp)

Estimations :
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
  Slip : moyenne=0,092 (vraie: 0,1)
  Guess : moyenne=0,212 (vraie: 0,2)
  P(competence) : moyenne=0,650 (vraie: 0.6)


In [16]:
// Visualisation du graphe avec priors informatifs
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_11_55_86.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Beta(2,18)[mean=0,1] 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 slipInformatif 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node3 -->
 
 node3 
 
 Bernoulli 
 
<!-- node2->node3 -->
 
 node2->node3 
 
 
 probTrue 
 
<!-- node4 -->
 
 node4 
 
 vbool68[observation2] 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 
<!-- node5 -->
 
 node5 
 
 Not 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 b 
 
<!-- node6 -->
 
 node6 
 
 reponse2[observation2] 
 
<!-- node5->node6 -->
 
 node5->node6 
 
 
 
<!-- node7 -->
 
 node7 
 
 competence2[observation2] 
 
<!-- node7->node6 -->
 
 node7->node6 
 
 
 condition 
 
<!-- node8 -->
 
 node8 
 
 guessInformatif 
 
<!-- node9 -->
 
 node9 
 
 Bernoulli 
 
<!-- node8->node9 -->
 
 node8->node9 
 
 
 probTrue 
 
<!-- node9->node6 -->
 
 node9->node6 
 
 
 
<!-- node10 -->
 
 node10 
 
 Beta(3,12)[mean=0,2] 
 
<!-- node11 -->
 
 node11 
 
 Random 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 dist 
 
<!-- node11->node8 -->
 
 node11->node8 
 
 
 
<!-- node12 -->
 
 node12 
 
 pCompInformatif 
 
<!-- node13 -->
 
 node13 
 
 Bernoulli 
 
<!-- node12->node13 -->
 
 node12->node13 
 
 
 probTrue 
 
<!-- node13->node7 -->
 
 node13->node7 
 
 
 
<!-- node14 -->
 
 node14 
 
 Beta(6,4)[mean=0,6] 
 
<!-- node15 -->
 
 node15 
 
 Random 
 
<!-- node14->node15 -->
 
 node14->node15 
 
 
 dist 
 
<!-- node15->node12 -->
 
 node15->node12


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Comparaison avec priors informatifs

Le graphe a la meme structure que le precedent, mais les **priors** sont maintenant informatifs :

| Parametre | Prior uniforme | Prior informatif |
|-----------|----------------|------------------|
| **slip** | Beta(1,1) = U(0,1) | Beta(2,18) centre sur 0.1 |
| **guess** | Beta(1,1) = U(0,1) | Beta(3,12) centre sur 0.2 |
| **pComp** | Beta(1,1) = U(0,1) | Beta(6,4) centre sur 0.6 |

**Impact sur l'inference** : Les priors informatifs agissent comme des "pseudo-observations" qui guident l'estimation vers des valeurs realistes, brisant la symetrie du probleme d'identifiabilite.

### Impact des priors informatifs

L'utilisation de priors informatifs (Beta(2,18) pour slip, Beta(3,12) pour guess) permet de :

1. **Regulariser** les estimations vers des valeurs realistes
2. **Briser la symetrie** du probleme d'identifiabilite
3. **Incorporer les connaissances du domaine** (slip et guess sont generalement faibles)

| Configuration | Slip estime | Guess estime | Fiabilite |
|---------------|-------------|--------------|-----------|
| Priors uniformes Beta(1,1) | ~0.33 | ~0.67 | Faible |
| Priors informatifs | Plus proche de 0.1 | Plus proche de 0.2 | Meilleure |

> **Bonne pratique** : En psychometrie, les priors pour slip et guess sont souvent contraints a etre inferieurs a 0.3, ce qui correspond a la realite empirique des tests bien concus.

## 7. Comparaison IRT vs DINA

| Aspect | IRT | DINA |
|--------|-----|------|
| **Capacite** | Continue, unidimensionnelle | Discrete, multidimensionnelle |
| **Interpretation** | "Niveau global" | "Competences specifiques" |
| **Complexite** | Simple, robuste | Plus complexe, informatif |
| **Utilisation** | Tests standardises | Diagnostic pedagogique |
| **Inference** | Plus facile | Necessite matrice Q |

### Quand utiliser chaque modele ?

| Scenario | Modele recommande | Justification |
|----------|-------------------|---------------|
| Test standardise (SAT, GMAT) | **IRT** | Une seule dimension mesuree, grand echantillon |
| Diagnostic pedagogique | **DINA** | Identifier les competences manquantes |
| Certification professionnelle | **DINA** | Verifier les prerequis specifiques |
| Classement de joueurs | **TrueSkill** (prochain notebook) | Competences relatives, matchs |

### Formulation mathematique comparee

**IRT (2PL)** :
$$P(\text{correct}) = \frac{1}{1 + e^{-a(\theta - b)}}$$

ou $\theta$ = capacite, $b$ = difficulte, $a$ = discrimination

**DINA** :
$$P(\text{correct}) = (1-s)^{\eta} \cdot g^{1-\eta}$$

ou $\eta = \prod_{k} \alpha_k^{q_{jk}}$ (1 si toutes competences requises, 0 sinon), $s$ = slip, $g$ = guess

## 8. Exercice : Evaluer un Nouvel Etudiant

### Enonce

Un nouvel etudiant passe un test de 5 questions. Le test evalue 2 competences :
- Q1, Q2 : Competence 1 seulement
- Q3, Q4 : Competence 2 seulement
- Q5 : Competences 1 ET 2

Resultats : Q1=correct, Q2=correct, Q3=incorrect, Q4=correct, Q5=incorrect

**Question** : Quelles sont les probabilites que l'etudiant possede C1 et C2 ?

### Implementation de l'exercice

Le code suivant construit un modele DINA pour le scenario decrit :

| Question | Competences | Resultat |
|----------|-------------|----------|
| Q1 | C1 | Correct |
| Q2 | C1 | Correct |
| Q3 | C2 | Incorrect |
| Q4 | C2 | Correct |
| Q5 | C1 ET C2 | Incorrect |

Les parametres slip=0.1 et guess=0.15 sont fixes (valeurs typiques en psychometrie).

> **Prediction intuitive** : Avec 2/2 sur C1 et 1/2 sur C2, on s'attend a P(C1) eleve et P(C2) incertain. L'echec a Q5 devrait trancher en defaveur de C2.

In [17]:
// EXERCICE : Evaluation d'un nouvel etudiant

Variable<bool> c1 = Variable.Bernoulli(0.5).Named("C1");
Variable<bool> c2 = Variable.Bernoulli(0.5).Named("C2");

double slip = 0.1;
double guess = 0.15;

// Q1 : C1 seulement -> correct
Variable<bool> rQ1 = Variable.New<bool>().Named("Q1");
using (Variable.If(c1)) { rQ1.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c1)) { rQ1.SetTo(Variable.Bernoulli(guess)); }
rQ1.ObservedValue = true;

// Q2 : C1 seulement -> correct
Variable<bool> rQ2 = Variable.New<bool>().Named("Q2");
using (Variable.If(c1)) { rQ2.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c1)) { rQ2.SetTo(Variable.Bernoulli(guess)); }
rQ2.ObservedValue = true;

// Q3 : C2 seulement -> incorrect
Variable<bool> rQ3 = Variable.New<bool>().Named("Q3");
using (Variable.If(c2)) { rQ3.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c2)) { rQ3.SetTo(Variable.Bernoulli(guess)); }
rQ3.ObservedValue = false;

// Q4 : C2 seulement -> correct
Variable<bool> rQ4 = Variable.New<bool>().Named("Q4");
using (Variable.If(c2)) { rQ4.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c2)) { rQ4.SetTo(Variable.Bernoulli(guess)); }
rQ4.ObservedValue = true;

// Q5 : C1 ET C2 -> incorrect
Variable<bool> c1etc2 = (c1 & c2).Named("C1_ET_C2");
Variable<bool> rQ5 = Variable.New<bool>().Named("Q5");
using (Variable.If(c1etc2)) { rQ5.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c1etc2)) { rQ5.SetTo(Variable.Bernoulli(guess)); }
rQ5.ObservedValue = false;

InferenceEngine mEx = new InferenceEngine();
mEx.Compiler.CompilerChoice = CompilerChoice.Roslyn;
mEx.ShowFactorGraph = true;  // Genere un fichier .gv pour visualisation

Console.WriteLine("=== Evaluation du nouvel etudiant ===");
Console.WriteLine("Resultats : Q1=T, Q2=T, Q3=F, Q4=T, Q5=F\n");
Console.WriteLine($"P(C1) = {mEx.Infer<Bernoulli>(c1).GetProbTrue():F3}");
Console.WriteLine($"P(C2) = {mEx.Infer<Bernoulli>(c2).GetProbTrue():F3}");
Console.WriteLine($"\nInterpretation :");
Console.WriteLine("- L'etudiant a probablement C1 (2/2 correct sur questions C1)");
Console.WriteLine("- C2 est moins certain (1/2 correct, et Q5 echoue)");

=== Evaluation du nouvel etudiant ===
Resultats : Q1=T, Q2=T, Q3=F, Q4=T, Q5=F

Compiling model...done.
P(C1) = 0,958
P(C2) = 0,091

Interpretation :
- L'etudiant a probablement C1 (2/2 correct sur questions C1)
- C2 est moins certain (1/2 correct, et Q5 echoue)


### Analyse du diagnostic de l'etudiant

**Resultats** :
- P(C1) = **0.958** : L'etudiant a tres probablement C1
- P(C2) = **0.091** : L'etudiant n'a probablement PAS C2

**Raisonnement du modele** :

| Observation | Impact sur C1 | Impact sur C2 |
|-------------|---------------|---------------|
| Q1=T (C1) | Forte hausse | - |
| Q2=T (C1) | Confirmation | - |
| Q3=F (C2) | - | Baisse |
| Q4=T (C2) | - | Legere hausse |
| Q5=F (C1 ET C2) | Legere baisse | Forte baisse |

**Cle** : L'echec a Q5 (C1 ET C2) avec P(C1) eleve implique que C2 est probablement manquant (explaining away).

> **Application pedagogique** : Ce diagnostic permet de cibler la remediation sur C2 specifiquement, plutot que de faire reprendre tout le cours a l'etudiant.

### Graphe de facteurs de l'exercice diagnostic

Ce graphe montre le **modele DINA complet** pour l'evaluation de l'etudiant :

| Question | Variables connectees | Observation |
|----------|---------------------|-------------|
| Q1, Q2 | C1 uniquement | true, true |
| Q3, Q4 | C2 uniquement | false, true |
| Q5 | C1 AND C2 | false |

**Propagation des messages** :

1. Q1=T et Q2=T → **forte evidence** pour C1 (deux confirmations)
2. Q3=F et Q4=T → **evidence ambigue** pour C2 (une erreur, une reussite)
3. Q5=F avec C1 probable → **evidence contre C2** (explaining away)

Le graphe visualise pourquoi P(C1)=0.96 >> P(C2)=0.09 : les messages provenant de Q5 "expliquent" l'echec par l'absence de C2 plutot que de C1.

In [18]:
// Visualisation du graphe de facteurs de l'exercice
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_11_56_65.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Bernoulli(0,5) 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 C1 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node5 -->
 
 node5 
 
 True 
 
<!-- node2->node5 -->
 
 node2->node5 
 
 
 condition 
 
<!-- node10 -->
 
 node10 
 
 True 
 
<!-- node2->node10 -->
 
 node2->node10 
 
 
 condition 
 
<!-- node13 -->
 
 node13 
 
 And 
 
<!-- node2->node13 -->
 
 node2->node13 
 
 
 a 
 
<!-- node3 -->
 
 node3 
 
 Bernoulli(0,9) 
 
<!-- node4 -->
 
 node4 
 
 Random 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 dist 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 
<!-- node6 -->
 
 node6 
 
 Bernoulli(0,15) 
 
<!-- node7 -->
 
 node7 
 
 Random 
 
<!-- node6->node7 -->
 
 node6->node7 
 
 
 dist 
 
<!-- node7->node5 -->
 
 node7->node5 
 
 
 
<!-- node8 -->
 
 node8 
 
 Bernoulli(0,9) 
 
<!-- node9 -->
 
 node9 
 
 Random 
 
<!-- node8->node9 -->
 
 node8->node9 
 
 
 dist 
 
<!-- node9->node10 -->
 
 node9->node10 
 
 
 
<!-- node11 -->
 
 node11 
 
 Bernoulli(0,15) 
 
<!-- node12 -->
 
 node12 
 
 Random 
 
<!-- node11->node12 -->
 
 node11->node12 
 
 
 dist 
 
<!-- node12->node10 -->
 
 node12->node10 
 
 
 
<!-- node15 -->
 
 node15 
 
 C1_ET_C2 
 
<!-- node13->node15 -->
 
 node13->node15 
 
 
 
<!-- node14 -->
 
 node14 
 
 C2 
 
<!-- node14->node13 -->
 
 node14->node13 
 
 
 b 
 
<!-- node20 -->
 
 node20 
 
 False 
 
<!-- node14->node20 -->
 
 node14->node20 
 
 
 condition 
 
<!-- node25 -->
 
 node25 
 
 True 
 
<!-- node14->node25 -->
 
 node14->node25 
 
 
 condition 
 
<!-- node30 -->
 
 node30 
 
 False 
 
<!-- node15->node30 -->
 
 node15->node30 
 
 
 condition 
 
<!-- node16 -->
 
 node16 
 
 Bernoulli(0,5) 
 
<!-- node17 -->
 
 node17 
 
 Random 
 
<!-- node16->node17 -->
 
 node16->node17 
 
 
 dist 
 
<!-- node17->node14 -->
 
 node17->node14 
 
 
 
<!-- node18 -->
 
 node18 
 
 Bernoulli(0,9) 
 
<!-- node19 -->
 
 node19 
 
 Random 
 
<!-- node18->node19 -->
 
 node18->node19 
 
 
 dist 
 
<!-- node19->node20 -->
 
 node19->node20 
 
 
 
<!-- node21 -->
 
 node21 
 
 Bernoulli(0,15) 
 
<!-- node22 -->
 
 node22 
 
 Random 
 
<!-- node21->node22 -->
 
 node21->node22 
 
 
 dist 
 
<!-- node22->node20 -->
 
 node22->node20 
 
 
 
<!-- node23 -->
 
 node23 
 
 Bernoulli(0,9) 
 
<!-- node24 -->
 
 node24 
 
 Random 
 
<!-- node23->node24 -->
 
 node23->node24 
 
 
 dist 
 
<!-- node24->node25 -->
 
 node24->node25 
 
 
 
<!-- node26 -->
 
 node26 
 
 Bernoulli(0,15) 
 
<!-- node27 -->
 
 node27 
 
 Random 
 
<!-- node26->node27 -->
 
 node26->node27 
 
 
 dist 
 
<!-- node27->node25 -->
 
 node27->node25 
 
 
 
<!-- node28 -->
 
 node28 
 
 Bernoulli(0,9) 
 
<!-- node29 -->
 
 node29 
 
 Random 
 
<!-- node28->node29 -->
 
 node28->node29 
 
 
 dist 
 
<!-- node29->node30 -->
 
 node29->node30 
 
 
 
<!-- node31 -->
 
 node31 
 
 Bernoulli(0,15) 
 
<!-- node32 -->
 
 node32 
 
 Random 
 
<!-- node31->node32 -->
 
 node31->node32 
 
 
 dist 
 
<!-- node32->node30 -->
 
 node32->node30


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Exercice supplementaire : Conception d'un test diagnostic

Imaginez que vous devez concevoir un test pour evaluer 3 competences en programmation :
- **C1** : Syntaxe de base (variables, boucles)
- **C2** : Structures de donnees (tableaux, listes)
- **C3** : Algorithmes (tri, recherche)

**Questions** :

1. Combien de questions minimum faut-il pour diagnostiquer chaque competence de maniere fiable ?

2. Proposez une matrice Q pour 6 questions qui permette de distinguer les 8 profils possibles d'etudiants (2^3 combinaisons de competences).

3. Si un etudiant echoue uniquement sur les questions necessitant C3, quelle remediation proposeriez-vous ?

> **Indice** : Pour distinguer tous les profils, il faut au moins une question testant chaque competence isolement (Q1-C1, Q2-C2, Q3-C3) et des questions composites pour valider les interactions.

## 9. Resume

| Concept | Description |
|---------|-------------|
| **IRT** | Modele capacite-difficulte pour tests unidimensionnels |
| **DINA** | Modele a competences discretes multiples |
| **Matrice Q** | Definition des competences requises par question |
| **Slip** | Erreur d'inattention (correct -> incorrect) |
| **Guess** | Reponse au hasard (incorrect -> correct) |
| **Many-to-many** | Questions multiples, competences multiples |

---

## Prochaine etape

Dans [Infer-6-TrueSkill](Infer-6-TrueSkill.ipynb), nous explorerons :

- Le systeme de classement TrueSkill (Xbox Live)
- L'apprentissage en ligne des competences
- La gestion des matchs nuls
- La visualisation du message passing

### Points cles a retenir

1. **IRT** : Ideal pour les tests standardises avec une dimension principale (intelligence, aptitude verbale)

2. **DINA** : Plus adapte au diagnostic pedagogique car il identifie les competences specifiques manquantes

3. **Slip et Guess** : Ces parametres de bruit sont difficiles a estimer sans priors informatifs ou donnees abondantes

4. **Matrice Q** : La conception du test (quelles competences pour chaque question) est cruciale pour le diagnostic

5. **Inference bayesienne** : Permet de quantifier l'incertitude sur les estimations et d'integrer les connaissances prealables

> **Perspective** : Ces modeles sont utilises en production par des plateformes educatives (Duolingo, Khan Academy) pour personnaliser les parcours d'apprentissage.

In [19]:
// Nettoyage des fichiers generes (optionnel)
int cleaned = FactorGraphHelper.CleanupGeneratedFiles();
Console.WriteLine($"Fichiers .gv et .svg nettoyes : {cleaned}");

Fichiers .gv et .svg nettoyes : 54
